## Log Linear Regression Assignment

In [1]:
# Load required libraries
%matplotlib inline
import category_encoders as ce
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_columns', 50)

In [2]:
# Load training set data
train_set = pd.read_csv(r'C:\Users\nayan\Downloads\caterpillar-tube-pricing\data\competition_data\train_set.csv')

train_set.head()

,tube_assembly_id,supplier,quote_date,annual_usage,min_order_quantity,bracket_pricing,quantity,cost
0,TA-00002,S-0066,2013-07-07,0,0,Yes,1,21.91
1,TA-00002,S-0066,2013-07-07,0,0,Yes,2,12.34
2,TA-00002,S-0066,2013-07-07,0,0,Yes,5,6.60
3,TA-00002,S-0066,2013-07-07,0,0,Yes,10,4.69
4,TA-00002,S-0066,2013-07-07,0,0,Yes,25,3.54


In [4]:
# Load test set data
test_set = pd.read_csv(r'C:\Users\nayan\Downloads\caterpillar-tube-pricing\data\competition_data\test_set.csv')

test_set.head()

,id,tube_assembly_id,supplier,quote_date,annual_usage,min_order_quantity,bracket_pricing,quantity
0,1,TA-00001,S-0066,2013-06-23,0,0,Yes,1
1,2,TA-00001,S-0066,2013-06-23,0,0,Yes,2
2,3,TA-00001,S-0066,2013-06-23,0,0,Yes,5
3,4,TA-00001,S-0066,2013-06-23,0,0,Yes,10
4,5,TA-00001,S-0066,2013-06-23,0,0,Yes,25


In [12]:
# Load tube data set
tube_set = pd.read_csv(r'C:\Users\nayan\Downloads\caterpillar-tube-pricing\data\competition_data\tube.csv')

tube_set.head()

,tube_assembly_id,material_id,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,end_x_2x,end_a,end_x,num_boss,num_bracket,other
0,TA-00001,SP-0035,12.70,1.65,164.00,5,38.10,N,N,N,N,EF-003,EF-003,0,0,0
1,TA-00002,SP-0019,6.35,0.71,137.00,8,19.05,N,N,N,N,EF-008,EF-008,0,0,0
2,TA-00003,SP-0019,6.35,0.71,127.00,7,19.05,N,N,N,N,EF-008,EF-008,0,0,0
3,TA-00004,SP-0019,6.35,0.71,137.00,9,19.05,N,N,N,N,EF-008,EF-008,0,0,0
4,TA-00005,SP-0029,19.05,1.24,109.00,4,50.80,N,N,N,N,EF-003,EF-003,0,0,0


In [15]:
# Load bill of materials data set
b_o_m_set = pd.read_csv(r'C:\Users\nayan\Downloads\caterpillar-tube-pricing\data\competition_data\bill_of_materials.csv')

b_o_m_set.head()

,tube_assembly_id,component_id_1,quantity_1,component_id_2,quantity_2,component_id_3,quantity_3,component_id_4,quantity_4,component_id_5,quantity_5,component_id_6,quantity_6,component_id_7,quantity_7,component_id_8,quantity_8
0,TA-00001,C-1622,2.00,C-1629,2.00,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan
1,TA-00002,C-1312,2.00,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan
2,TA-00003,C-1312,2.00,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan
3,TA-00004,C-1312,2.00,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan
4,TA-00005,C-1624,1.00,C-1631,1.00,C-1641,1.00,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan


In [16]:
# Load components data set
comps_set = pd.read_csv(r'C:\Users\nayan\Downloads\caterpillar-tube-pricing\data\competition_data\components.csv')

comps_set.head()

,component_id,name,component_type_id
0,9999,OTHER,OTHER
1,C-0001,SLEEVE,CP-024
2,C-0002,SLEEVE,CP-024
3,C-0003,SLEEVE-FLARED,CP-024
4,C-0004,NUT,CP-026


In [18]:
# Convert date/time field to pandas date/time
train_set['quote_date'] = pd.to_datetime(train_set['quote_date'], infer_datetime_format=True)
test_set['quote_date'] = pd.to_datetime(test_set['quote_date'], infer_datetime_format=True)

In [ ]:
# Create function to  wrangle data
def wrangle(X):
    